In [1]:
%load_ext autoreload
%autoreload 2

In [7]:
import os 
os.chdir('/Users/egill/dev/genki/genki-signals')

import time
from threading import Thread

from genki_signals.buffers import DataBuffer
from genki_signals.signal_sources import WaveSignalSource
import genki_signals.signal_functions as sf
from genki_signals.signal_system import SignalSystem

In [8]:
wave_source = WaveSignalSource(ble_address='5C72E785-DAF5-1E32-599D-EBF56B8ECD5B')

In [39]:
def respond(cls_idx):
    if cls_idx == 1:
        print("Left!\n")
    elif cls_idx == 2:
        print("Right!\n")
        


derived = [
    sf.Concatenate(['gyro', 'acc'], name='model_input'),
    
    sf.WindowedInference(
        'genki_signals/models/swipe_model/model.onnx', 
        'model_input',
        name='swipe_inference', 
        window_size=128, 
        window_overlap=96, 
        output_shape=(3,)),
    sf.ObjectTracker('swipe_inference', name='tracker', output_shape=(3,), callback=respond, window_size=16)
]

system = SignalSystem(wave_source, derived_signals=derived)
buffer = DataBuffer(maxlen=400)

def run_system():
    system.start()
    while True:
        buffer.extend(system.read())
        time.sleep(1 / 50)
        if not system.is_active:
            return

t = Thread(target=run_system)
t.start()

Connecting to wave at address 5C72E785-DAF5-1E32-599D-EBF56B8ECD5B
Connected to Wave
Left!

Left!

Left!

Right!



In [50]:
system.stop(); t.join()

Got a cancel message, exiting.


In [43]:
buffer.plot('swipe_inference')

Figure(axes=[Axis(label='t', scale=LinearScale()), Axis(label='swipe_inference', orientation='vertical', scale…